In [1]:
#skip warning
import warnings
warnings.filterwarnings('ignore')
#connect to drive
from google.colab import drive  
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install PyMuPDF #package to read pdf
!pip install nltk==3.4 #package to preprocessing text
import nltk 
nltk.download("all") #download all tools in nltk

     |████████████████████████████████| 6.4MB 6.9MB/s 
     |████████████████████████████████| 1.4MB 7.6MB/s 
  Created wheel for nltk: filename=nltk-3.4-cp37-none-any.whl size=1436385 sha256=9fb9df9bd106fb7bd607ec1619d029292ee8789795ca3ec41fe333abdcff5f93
  Stored in directory: /root/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [3]:
import numpy as np #math and matrix processing
import pandas as pd #csv/dataframe processing
import fitz #lib to read pdf(pyMUPDF)
import re #regex lib
from nltk.tokenize import sent_tokenize #tokenisasi kalimat
from nltk.tokenize import word_tokenize #memisahkan antar kata
from nltk import ngrams #pembentuk token
from nltk.corpus import stopwords #penghapus kata tidak penting

In [22]:
list_pdf = pd.read_csv('/content/drive/MyDrive/Skripsi_Putri/DataEmiten_2018.csv') #baca nama-nama perusahaan
sektor = 'IDXPROPERT' #set sektor
tahun = '2018'

In [23]:
#fungsi baca pdf
def get_dataset(sektor,pdfs) :
  docs = []
  list_file = []
  list_companies = pdfs[pdfs["Sektor"] == sektor]
  for code in list_companies['Code'] :
    pdf_path = "/content/drive/MyDrive/Skripsi_Putri/Annual Report Perusahaan/%s_%s/%s_%s.pdf"%(sektor,tahun,code,tahun)
    try :
      doc = fitz.open(pdf_path)
    except :
      print("%s failed to extract"%code)
      continue
    num_of_pages = doc.pageCount
    print("Extract %s.pdf, num_of_page = %s"%(code,num_of_pages))
    text = doc.loadPage(0).getText('text')
    for j in range(1,num_of_pages) :
      page = doc.loadPage(j) 
      text_temp = page.getText("text")
      text = text + '. ' + text_temp
    docs.append(text)
    print("Extract %s.pdf is finished"%(code))
    list_file.append(code)
  return docs,list_file

In [24]:
#fungsi cleaning text
def data_clean(dataset) :
  docs = dataset.copy() # dicopy biar tidak menimpa satu sama lain
  result = []
  for data in docs :
    data = re.sub('\n', ' ', data) #ganti newline dengan spasi (textnya tidak newline)
    data = data.replace('&','and') #ganti semua & dengan and
    data = data.replace('/',' or ') #ganti semua / dengan or
    data = re.sub('[^a-zA-Z0-9.]',' ',data) #hapus non-alfanumerik selain titik
    data = re.sub(' +', ' ', data) #hapus spasi berlebih
    data = data.replace(' .','.') #hapus spasi yang setelahnya ada titik. (makan . -> makan.)
    data = re.sub('\.+','.',data) #hapus titik berlebih
    data = data.lower() #mengubah semua huruf jadi lowercase
    result.append(data) #hasil cleaning
  return result

#fungsi tokenisasi
def sentence_tokenize(dataset) :  
  docs = dataset.copy() 
  result = []
  for data in docs :
    sentences = sent_tokenize(data) #proses memisahkan antar kalimat
    result.append(sentences) #hasil pemisahaan
  return result

#pembentuk token
def ngram(dataset,n) : # n : banyaknya kata dalam satu token
  docs = dataset.copy()
  result = []
  for sentences in docs :
    token = []
    for sent in sentences :
      tokenize = word_tokenize(sent) #proses tokenisasi
      filtered_sentence = []  
      nltk_stop_words = set(stopwords.words("english"))
      for w in tokenize:  
        if w not in nltk_stop_words: #hapus kata yang tidak begitu penitng
          filtered_sentence.append(w)
     
      #menbetuk token
      gram = list(ngrams(filtered_sentence, n))
      gram = [' '.join(token) for token in gram]
      token += gram  
    result.append(token) #simpan
  return result

#membersihkan ulang hasil tokenisasi
def token_clean(dataset) :
  docs = dataset.copy()
  result = []
  for tokens in docs :
    for i in range(len(tokens)) :
      tokens[i] = re.sub('\.|\. | \.',"",tokens[i]) #menghapus titik, titik kemudian ada spasi, spasi kemudian ada titik
    result.append(tokens) #save
  return result

#clean dan gabungkan unigram, bigram dan trigram
def clean_join(uni,bi,tri) :
  #pembersihan token
  uni = token_clean(uni)
  bi = token_clean(bi)
  tri = token_clean(tri)

  #hapus non-alfabet token. contoh : Rp10 million
  for i in range(len(uni)) :
    uni[i] = [word for word in uni[i] if word.replace(' ','').isalpha()]
  for i in range(len(bi)) :
    bi[i] = [word for word in bi[i] if word.replace(' ','').isalpha()]
  for i in range(len(tri)) :
    tri[i] = [word for word in tri[i] if word.replace(' ','').isalpha()]

  #menggabungkan unigram, bigram, dan trigram
  all_token = []
  num_of_docs = len(files)
  for i in range(num_of_docs) :
    all_token.append(bi[i]+uni[i]+tri[i])
  return all_token

In [25]:
dataset,files = get_dataset(sektor, list_pdf) #baca pdf 
print("list_files =",files) #cetak list pdf yang terbaca

Extract APLN.pdf, num_of_page = 381
Extract APLN.pdf is finished
Extract ARMY.pdf, num_of_page = 149
Extract ARMY.pdf is finished
Extract ASRI.pdf, num_of_page = 266
Extract ASRI.pdf is finished
Extract BCIP.pdf, num_of_page = 129
Extract BCIP.pdf is finished
Extract BEST.pdf, num_of_page = 157
Extract BEST.pdf is finished
Extract BIKA.pdf, num_of_page = 265
Extract BIKA.pdf is finished
Extract BIPP.pdf, num_of_page = 196
Extract BIPP.pdf is finished
Extract BKDP.pdf, num_of_page = 24
Extract BKDP.pdf is finished
Extract BKSL.pdf, num_of_page = 237
Extract BKSL.pdf is finished
Extract BSDE.pdf, num_of_page = 348
Extract BSDE.pdf is finished
Extract CITY.pdf, num_of_page = 145
Extract CITY.pdf is finished
Extract COWL.pdf, num_of_page = 151
Extract COWL.pdf is finished
Extract CPRI.pdf, num_of_page = 64
Extract CPRI.pdf is finished
Extract CSIS.pdf, num_of_page = 177
Extract CSIS.pdf is finished
Extract CTRA.pdf, num_of_page = 354
Extract CTRA.pdf is finished
Extract CTRR.pdf, num_of_pa

mupdf: object out of range (1126 0 R); xref size 1126
mupdf: object out of range (1126 0 R); xref size 1126
mupdf: object out of range (1127 0 R); xref size 1126
mupdf: object out of range (1127 0 R); xref size 1126


Extract SMRA.pdf, num_of_page = 313
Extract SMRA.pdf is finished
Extract TARA.pdf, num_of_page = 168
Extract TARA.pdf is finished
Extract URBN.pdf, num_of_page = 204
Extract URBN.pdf is finished
list_files = ['APLN', 'ARMY', 'ASRI', 'BCIP', 'BEST', 'BIKA', 'BIPP', 'BKDP', 'BKSL', 'BSDE', 'CITY', 'COWL', 'CPRI', 'CSIS', 'CTRA', 'CTRR', 'DART', 'DILD', 'DMAS', 'DUTI', 'ELTY', 'EMDE', 'FMII', 'FORZ', 'GAMA', 'GMTD', 'GPRA', 'GWSA', 'INPP', 'JRPT', 'KIJA', 'LAND', 'LCGP', 'LPCK', 'LPKR', 'LPLI', 'MDLN', 'MPRO', 'MTLA', 'MTSM', 'MYRX', 'NIRO', 'OMRE', 'PLIN', 'POLI', 'POLL', 'PPRO', 'PTHK', 'PUDP', 'PWON', 'RBMS', 'RDTX', 'RIMO', 'RODA', 'SATU', 'SCBD', 'SMDM', 'SMRA', 'TARA', 'URBN']


In [26]:
dataset2 = data_clean(dataset) #clean text
dataset2 = sentence_tokenize(dataset2) #pemisah antar kalimat

In [19]:
token_unigram = ngram(dataset2,1) #pembentuk token 1 word
token_bigram = ngram(dataset2,2) #pembentuk token 2 word
token_trigram = ngram(dataset2,3) #pembentuk token 3 word

In [20]:
pdf_tokens = clean_join(token_unigram,token_bigram,token_trigram) #membersihkan dan menggabungkan token

In [21]:
#save token to drive per sektor
import json
save_tokens = dict()
for i in range(len(pdf_tokens)) :
  save_tokens[files[i]] = pdf_tokens[i]
with open('/content/drive/MyDrive/Skripsi_Putri/Hasil/Tokens/'+sektor+'_'+tahun+'.json', 'w') as fp:
  json.dump(save_tokens, fp)